In [2]:
import asyncio
import logging
import sys
import os

from astropy.coordinates import Angle
from astropy.time import Time
import astropy.units as u

from lsst.ts import salobj
from lsst.ts.observatory.control.maintel.mtcs import MTCS
from lsst.ts.observatory.control.maintel.comcam import ComCam

#import mt_utils

In [3]:
print(os.environ["OSPL_URI"])

file:///opt/lsst/src/ts_ddsconfig/config/ospl-shmem-debug.xml


In [4]:
#os.environ["OSPL_URI"]="file:///home/bstalder/ts_ddsconfig/config/ospl-sp.xml"
os.environ["LSST_DDS_RESPONSIVENESS_TIMEOUT"]="15s"

In [5]:
stream_handler = logging.StreamHandler(sys.stdout)

logger = logging.getLogger()
logger.addHandler(stream_handler)
logger.level = logging.DEBUG

In [6]:
domain = salobj.Domain()

await asyncio.sleep(10)

#mtcs = MTCS(domain)
comcam = ComCam(domain)
#mtptg = salobj.Remote(domain, "MTPtg")
#newmtmount = salobj.Remote(domain, "NewMTMount")

#await asyncio.gather(mtcs.start_task, comcam.start_task)
await asyncio.gather(comcam.start_task) #, mtptg.start_task, newmtmount.start_task)

cccamera: Adding all resources.
ccheaderservice: Adding all resources.
ccarchiver: Adding all resources.
Read historical data in 0.00 sec
Read 1 history items for RemoteEvent(CCHeaderService, 0, authList)
Read 100 history items for RemoteEvent(CCHeaderService, 0, heartbeat)
Read 50 history items for RemoteEvent(CCHeaderService, 0, largeFileObjectAvailable)
Read 1 history items for RemoteEvent(CCHeaderService, 0, logLevel)
Read 100 history items for RemoteEvent(CCHeaderService, 0, logMessage)
Read 1 history items for RemoteEvent(CCHeaderService, 0, simulationMode)
Read 1 history items for RemoteEvent(CCHeaderService, 0, softwareVersions)
Read 11 history items for RemoteEvent(CCHeaderService, 0, summaryState)
Read historical data in 0.03 sec
Read 1 history items for RemoteEvent(CCArchiver, 0, authList)
Read 100 history items for RemoteEvent(CCArchiver, 0, heartbeat)
Read 100 history items for RemoteEvent(CCArchiver, 0, imageInOODS)
Read 100 history items for RemoteEvent(CCArchiver, 0, im

[[None, None, None]]

In [ ]:
await comcam.enable(settings={"cccamera": "Normal",
                              "ccarchiver": "normal",
                              "ccheaderservice": None})

In [7]:
date = Time.now()
group_id = f'CALSET_{date.tai.strftime("%Y%m%d_%H%M")}'
#group_id = "CALSET_20200914_1449"
print(group_id)

CALSET_20210520_2313


In [8]:
#get available filters
await comcam.get_available_filters()

['i_06', 'r_03', 'g_07']

In [9]:
#get current filter
await comcam.get_current_filter()

'r_03'

In [10]:
# set filter manually
#old way
#comcam.rem.cccamera.cmd_setFilter.set(name="i_06")
#await comcam.rem.cccamera.cmd_setFilter.start(timeout=60)
#new way
await comcam.setup_instrument(filter="i_06")

Filter i_06 in position.


In [11]:
#for testing L3 condensation
await comcam.take_flats(exptime=1.0, nflats=1, group_id=group_id, note="test")#, filter="g_07")

imagetype: FLAT, skip TCS synchronization.
FLAT 0001 - 0001


array([2021052000001])

In [19]:
await comcam.take_bias(nbias=10, group_id=group_id, note="standardCal")

imagetype: BIAS, skip TCS synchronization.
BIAS 0001 - 0010
BIAS 0002 - 0010
BIAS 0003 - 0010
BIAS 0004 - 0010
BIAS 0005 - 0010
BIAS 0006 - 0010
BIAS 0007 - 0010
BIAS 0008 - 0010
BIAS 0009 - 0010
BIAS 0010 - 0010


array([2021051400005, 2021051400006, 2021051400007, 2021051400008,
       2021051400009, 2021051400010, 2021051400011, 2021051400012,
       2021051400013, 2021051400014])

In [20]:
await comcam.take_darks(exptime=100.0, ndarks=10, group_id=group_id, note="standardCal")

imagetype: DARK, skip TCS synchronization.
DARK 0001 - 0010
DARK 0002 - 0010
DARK 0003 - 0010
DARK 0004 - 0010
DARK 0005 - 0010
DARK 0006 - 0010
DARK 0007 - 0010
logMessage DDS read queue is filling: 17 of 100 elements
DARK 0008 - 0010
DARK 0009 - 0010
DARK 0010 - 0010
logMessage DDS read queue is filling: 17 of 100 elements


array([2021051400015, 2021051400016, 2021051400017, 2021051400018,
       2021051400019, 2021051400020, 2021051400021, 2021051400022,
       2021051400023, 2021051400024])

In [21]:
await comcam.take_flats(exptime=5.0, nflats=10, group_id=group_id, note="standardCal")

imagetype: FLAT, skip TCS synchronization.
FLAT 0001 - 0010
FLAT 0002 - 0010
FLAT 0003 - 0010
FLAT 0004 - 0010
FLAT 0005 - 0010
FLAT 0006 - 0010
FLAT 0007 - 0010
FLAT 0008 - 0010
FLAT 0009 - 0010
FLAT 0010 - 0010


array([2021051400025, 2021051400026, 2021051400027, 2021051400028,
       2021051400029, 2021051400030, 2021051400031, 2021051400032,
       2021051400033, 2021051400034])

In [ ]:
# FOR PTC run
await comcam.take_bias(nbias=5, group_id=group_id, note="PTC")
await comcam.take_flats(exptime=0.5, nflats=2, group_id=group_id, note="PTC")
await comcam.take_flats(exptime=1.0, nflats=2, group_id=group_id, note="PTC")
await comcam.take_flats(exptime=1.5, nflats=2, group_id=group_id, note="PTC")
await comcam.take_flats(exptime=2.0, nflats=2, group_id=group_id, note="PTC")
await comcam.take_flats(exptime=3.0, nflats=2, group_id=group_id, note="PTC")
await comcam.take_flats(exptime=4.0, nflats=2, group_id=group_id, note="PTC")
await comcam.take_flats(exptime=5.0, nflats=2, group_id=group_id, note="PTC")
await comcam.take_flats(exptime=6.0, nflats=2, group_id=group_id, note="PTC")
await comcam.take_flats(exptime=7.0, nflats=2, group_id=group_id, note="PTC")
await comcam.take_flats(exptime=8.0, nflats=2, group_id=group_id, note="PTC")
await comcam.take_flats(exptime=9.0, nflats=2, group_id=group_id, note="PTC")
await comcam.take_flats(exptime=10.0, nflats=2, group_id=group_id, note="PTC")
await comcam.take_flats(exptime=15.0, nflats=2, group_id=group_id, note="PTC")
await comcam.take_flats(exptime=20.0, nflats=2, group_id=group_id, note="PTC")
await comcam.take_flats(exptime=25.0, nflats=2, group_id=group_id, note="PTC")
await comcam.take_flats(exptime=30.0, nflats=2, group_id=group_id, note="PTC")
await comcam.take_flats(exptime=40.0, nflats=2, group_id=group_id, note="PTC")
await comcam.take_flats(exptime=50.0, nflats=2, group_id=group_id, note="PTC")
await comcam.take_flats(exptime=60.0, nflats=2, group_id=group_id, note="PTC")
await comcam.take_bias(nbias=5, group_id=group_id, note="PTC")

In [ ]:
# For gain stability run
await comcam.take_bias(nbias=5, group_id=group_id, note="gainStability")
for i in range(0,2000):
    await comcam.take_flats(exptime=21.0, nflats=2000, group_id=group_id, note="gainStability")
    await asyncio.sleep(10)
    
await comcam.take_bias(nbias=5, group_id=group_id, note="gainStability")

In [ ]:
await comcam.standby()

In [ ]:
await domain.close()